In [1]:
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
from InitKmeans import InitKmeans
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
     
def pre_processData(newsgroups_train):
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(newsgroups_train)):
        newsgroups_train[i] = newsgroups_train[i].lower()
        #newsgroups_train[i] = tokenizer.tokenize(newsgroups_train[i])
    #newsgroups_train = [[token for token in doc if not token.isdigit()] for doc in newsgroups_train]
    newsgroups_train = [doc.split(' ') for doc in newsgroups_train]
    return newsgroups_train
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
from InitKmeans import InitKmeans
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
import sys  
import json
if __name__ == "__main__":
    newsgroups_train = list()
    news_labels = {}
    docID_username = {}
    
    i = 0
    file_path="D:/part_news3_biterm.txt"
    with open(file_path) as fp:
        lines = fp.read().split("\n")
        for line in lines:
            if line:
                text = json.loads(line)["bitermText"].strip()
                label = json.loads(line)["clusterNo"]
                newsgroups_train.append(text)
                news_labels[i] = label
                i+=1
    fp.close()
    
    corpus = pre_processData(newsgroups_train)

D:\anacoda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
doc_biterm = {}
for docID in range(len(corpus)):
    words = corpus[docID]
    duplicate_biterm = []
    doc_biterm[docID] = {}
    for biterm in words:
        hash_biterm = set(biterm.split(','))
        if hash_biterm not in duplicate_biterm:
            duplicate_biterm.append(hash_biterm)
            doc_biterm[docID][biterm] = 1
        else:
            if biterm in doc_biterm[docID].keys():
                doc_biterm[docID][biterm] += 1
            else:
                doc_biterm[docID][biterm] = 1
doc_words = {}
i_biterm = 0
for docID in range(len(corpus)):
    doc_words[docID] = {}
    words = corpus[docID]
    for biterm in words:
        for word in biterm.split(','):
            if word not in doc_words[docID].keys():
                doc_words[docID][word] = 0
            doc_words[docID][word] += 1

In [16]:
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.003
    beta = 0.02
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    if batch_i != 1:
        start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    

start  0  end  1895 topics:  74 truth:  32  NMI:  0.8441538426522822
start  0  end  3790 topics:  106 truth:  62  NMI:  0.8312210474530084
start  1896  end  5685 topics:  90 truth:  48  NMI:  0.8134929015078751
start  3791  end  7580 topics:  87 truth:  38  NMI:  0.8254108499197133
start  5686  end  9475 topics:  94 truth:  44  NMI:  0.8316848199157013
start  7581  end  11370 topics:  84 truth:  33  NMI:  0.8465183329341434
start  9476  end  13265 topics:  55 truth:  23  NMI:  0.8861585753363685
start  11371  end  15160 topics:  61 truth:  35  NMI:  0.8907185578531914
start  13266  end  17055 topics:  71 truth:  37  NMI:  0.8914560693223879
start  15161  end  18950 topics:  57 truth:  31  NMI:  0.8855585610744889
start  17056  end  20845 topics:  55 truth:  28  NMI:  0.9122174573737886
start  18951  end  22740 topics:  52 truth:  22  NMI:  0.9094036828236365
start  20846  end  24635 topics:  50 truth:  21  NMI:  0.8864085759101779
start  22741  end  26530 topics:  68 truth:  30  NMI:  

In [3]:
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.3
    beta = 0.02
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    if batch_i != 1:
        start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    

start  0  end  1895 topics:  91 truth:  32  NMI:  0.8541620657728693
start  0  end  3790 topics:  133 truth:  62  NMI:  0.825823225627117
start  1896  end  5685 topics:  114 truth:  48  NMI:  0.8033470558632811
start  3791  end  7580 topics:  106 truth:  38  NMI:  0.8289650313973387
start  5686  end  9475 topics:  119 truth:  44  NMI:  0.8322720147139834
start  7581  end  11370 topics:  100 truth:  33  NMI:  0.8503644239682501
start  9476  end  13265 topics:  64 truth:  23  NMI:  0.8652697963710134
start  11371  end  15160 topics:  81 truth:  35  NMI:  0.9006097384737322
start  13266  end  17055 topics:  87 truth:  37  NMI:  0.8804840098696127
start  15161  end  18950 topics:  64 truth:  31  NMI:  0.8783485256081192
start  17056  end  20845 topics:  79 truth:  28  NMI:  0.8925488587618355
start  18951  end  22740 topics:  74 truth:  22  NMI:  0.8935566664477498
start  20846  end  24635 topics:  65 truth:  21  NMI:  0.8783042535008054
start  22741  end  26530 topics:  76 truth:  30  NMI

In [4]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print()

print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8541620657728693, 0.825823225627117, 0.8033470558632811, 0.8289650313973387, 0.8322720147139834, 0.8503644239682501, 0.8652697963710134, 0.9006097384737322, 0.8804840098696127, 0.8783485256081192, 0.8925488587618355, 0.8935566664477498, 0.8783042535008054, 0.8529155013334383, 0.7888824121272956, 0.7680583941160468]

0.8496194983720305


In [5]:
import timeit
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
time_iteration_list = []
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.3
    beta = 0.02
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    time_start = timeit.default_timer()
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    if batch_i != 1:
        start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    time_stop = timeit.default_timer()
    time_iteration_list.append(time_stop-time_start)
    print(time_iteration_list)

start  0  end  1895 topics:  93 truth:  32  NMI:  0.8493299044970227
[124.47662545532339]
start  0  end  3790 topics:  135 truth:  62  NMI:  0.8358267224037599
[124.47662545532339, 322.9079524862167]
start  1896  end  5685 topics:  111 truth:  48  NMI:  0.8176920153201213
[124.47662545532339, 322.9079524862167, 259.24636514990857]
start  3791  end  7580 topics:  104 truth:  38  NMI:  0.8268768354116884
[124.47662545532339, 322.9079524862167, 259.24636514990857, 243.3053607597542]
start  5686  end  9475 topics:  124 truth:  44  NMI:  0.8341988252151786
[124.47662545532339, 322.9079524862167, 259.24636514990857, 243.3053607597542, 277.14703576694296]
start  7581  end  11370 topics:  105 truth:  33  NMI:  0.840153174618594
[124.47662545532339, 322.9079524862167, 259.24636514990857, 243.3053607597542, 277.14703576694296, 238.6720100092275]
start  9476  end  13265 topics:  69 truth:  23  NMI:  0.8664766012830064
[124.47662545532339, 322.9079524862167, 259.24636514990857, 243.3053607597542, 

In [7]:
print(topic_batch.keys())
print(topic_batch.values())

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
dict_values([93, 135, 111, 104, 124, 105, 69, 81, 83, 67, 73, 66, 56, 75, 92, 93])


In [ ]:
0.8461431116282747

In [15]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print()

print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8490775422178058, 0.8360113596016167, 0.8017326375272005, 0.8269963983134461, 0.8259949715651002, 0.8469034211974069, 0.8689500165610579, 0.8880185467940986, 0.8879679257956725, 0.8556842818338339, 0.8924981532543471, 0.8898471837668972, 0.8751979807882151, 0.8591115412272363, 0.7716295524551399, 0.7626682731533194]

0.8461431116282747


In [10]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print()

print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8498076980818865, 0.7774663436307994, 0.777480416315458, 0.799597977152846, 0.8123893850338224, 0.7853681035349211, 0.8601579571732086, 0.8769340148293916, 0.8248206313398326, 0.8543233453659018, 0.8462780378479873, 0.827834189858882, 0.8621949972642189, 0.7796067956372406, 0.7501453688617773, 0.6809083067114148]

0.8103320980399742


In [17]:
print(NMI_batch)
print(beta)
print(NMI_batch.keys())
print(NMI_batch.values())

{1: 0.8441538426522822, 2: 0.8312210474530084, 3: 0.8134929015078751, 4: 0.8254108499197133, 5: 0.8316848199157013, 6: 0.8465183329341434, 7: 0.8861585753363685, 8: 0.8907185578531914, 9: 0.8914560693223879, 10: 0.8855585610744889, 11: 0.9122174573737886, 12: 0.9094036828236365, 13: 0.8864085759101779, 14: 0.8599767131213099, 15: 0.7995858837720357, 16: 0.7883701825543133}
0.02
dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
dict_values([0.8441538426522822, 0.8312210474530084, 0.8134929015078751, 0.8254108499197133, 0.8316848199157013, 0.8465183329341434, 0.8861585753363685, 0.8907185578531914, 0.8914560693223879, 0.8855585610744889, 0.9122174573737886, 0.9094036828236365, 0.8864085759101779, 0.8599767131213099, 0.7995858837720357, 0.7883701825543133])


In [20]:
nmi_batch = {}
topic_batch = {}
start = 0
end = 0
total_batch = None
if len(corpus) % 500 == 0:
    total_batch = int(len(corpus) / 500)
else:
    total_batch = int(len(corpus) / 500) +1

for batch in range(total_batch):
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.3
    beta = 0.2
    end = 500 * (batch + 1)
    if end > len(corpus):
        end = len(corpus)
    nmi_batch[batch] = 0
    topic_batch[batch] = 0
    
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            nmi_batch[batch] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    start = end 
 
print(nmi_batch.keys())
print(nmi_batch.values())

start  0  end  500 topics:  44 truth:  32  NMI:  0.880181128846321
start  500  end  1000 topics:  37 truth:  29  NMI:  0.8781310441089455
start  1000  end  1500 topics:  36 truth:  27  NMI:  0.8903626031124846
start  1500  end  2000 topics:  49 truth:  53  NMI:  0.8802891820550155
start  2000  end  2500 topics:  41 truth:  29  NMI:  0.8065787161554409
start  2500  end  3000 topics:  39 truth:  30  NMI:  0.8244414545681393
start  3000  end  3500 topics:  40 truth:  30  NMI:  0.8427443074941608
start  3500  end  4000 topics:  45 truth:  45  NMI:  0.8249955372910738
start  4000  end  4500 topics:  40 truth:  18  NMI:  0.8249857718905035
start  4500  end  5000 topics:  30 truth:  17  NMI:  0.8373118759546907
start  5000  end  5500 topics:  33 truth:  18  NMI:  0.8274495385773019
start  5500  end  6000 topics:  44 truth:  37  NMI:  0.8243009403391134
start  6000  end  6500 topics:  35 truth:  21  NMI:  0.8392714729131685
start  6500  end  7000 topics:  33 truth:  21  NMI:  0.855681903306815

In [23]:
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.3
    beta = 0.3
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    if batch_i != 1:
        start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    

start  0  end  1895 topics:  50 truth:  32  NMI:  0.8809833190237517
start  0  end  3790 topics:  68 truth:  62  NMI:  0.8439591343395921
start  1896  end  5685 topics:  50 truth:  48  NMI:  0.8411933507485957
start  3791  end  7580 topics:  46 truth:  38  NMI:  0.8662774155065565
start  5686  end  9475 topics:  65 truth:  44  NMI:  0.861500698458091
start  7581  end  11370 topics:  61 truth:  33  NMI:  0.8637970647294302
start  9476  end  13265 topics:  47 truth:  23  NMI:  0.9275081042154694
start  11371  end  15160 topics:  43 truth:  35  NMI:  0.9296569611245717
start  13266  end  17055 topics:  40 truth:  37  NMI:  0.9310156872776246
start  15161  end  18950 topics:  31 truth:  31  NMI:  0.9147819104407958
start  17056  end  20845 topics:  45 truth:  28  NMI:  0.9205557215382661
start  18951  end  22740 topics:  41 truth:  22  NMI:  0.9556500802946675
start  20846  end  24635 topics:  28 truth:  21  NMI:  0.9269834155693576
start  22741  end  26530 topics:  40 truth:  30  NMI:  0.

In [24]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8809833190237517, 0.8439591343395921, 0.8411933507485957, 0.8662774155065565, 0.861500698458091, 0.8637970647294302, 0.9275081042154694, 0.9296569611245717, 0.9310156872776246, 0.9147819104407958, 0.9205557215382661, 0.9556500802946675, 0.9269834155693576, 0.9162053955295711, 0.8181493364763252, 0.7977881339967929]
0.8872503580793412


In [22]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8776164167985246, 0.8364336026215384, 0.8427075406409203, 0.8633197450712946, 0.8445904054720953, 0.8744979689512565, 0.9231958326975366, 0.9177916950253654, 0.9323616880042681, 0.9277386915588987, 0.9378230070071573, 0.9469150565186208, 0.9204913463472311, 0.9038963198811201, 0.8096090638413559, 0.7857563666730738]
0.8840465466943912


In [8]:
alpha_NMI = {}
alpha_topics = {}
for alpha in np.arange(0.1,1.1,0.1):
    alpha = np.around(alpha,decimals=1)
    
    total_batch = 16
    NMI_batch = {}
    start = 0
    end = 0
    topic_batch = {}
    time_iteration_list = []
    for batch_i in range(1,17):
        if batch_i == 16:
            end = int(len(corpus))
        else:
            end = int(len(corpus) / total_batch) * batch_i

        docID_assign_z = {}
        m_z = {}
        n_z = {}
        n_w = {}
        n_b = {}
        Topics = []
        V = set()
        D = set()
        #alpha = 0.6
        beta = 0.02
        NMI_batch[batch_i] = 0
        topic_batch[batch_i] = 0

        time_start = timeit.default_timer()
        total_iter = 10
        for iter in range(total_iter):
            for docID in range(start,end):
                words = corpus[docID]
                D.discard(docID)
                if docID in docID_assign_z.keys():
                    before_k = docID_assign_z[docID]
                    m_z[before_k].discard(docID)
                    for biterm in words:
                        for word in biterm.split(','):
                            n_z[before_k][word] -= 1
                            n_w[before_k] -=1
                else:
                    before_k = -1
                if len(D) == 0 and len(V) == 0:
                    choose_k = 0
                    D.add(docID)
                    docID_assign_z[docID] = choose_k
                    if choose_k not in m_z.keys():
                        m_z[choose_k] = set()
                    m_z[choose_k].add(docID)
                    for biterm in words:
                        for word in biterm.split(','):
                            if choose_k not in n_w.keys():
                                n_w[choose_k] = 0
                            if choose_k not in n_z.keys():
                                n_z[choose_k] = {}
                            if word not in n_z[choose_k].keys():
                                n_z[choose_k][word] = 0
                            n_z[choose_k][word] += 1
                            n_w[choose_k] += 1
                            V.add(word)
                    if choose_k not in Topics:
                        Topics.append(choose_k)
                else:
                    log_pro = []
                    for k in Topics:
                        pro_k = len(m_z[k])
                        if pro_k != 0:
                            i = 0
                            for biterm in words:
                                each_word = biterm.strip().split(',')
                                for word in each_word:
                                    if word not in n_z[k].keys():
                                        n_z[k][word] = 0
                                for j in range(doc_biterm[docID][biterm]):
                                    pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                    i += 1

                        if pro_k == 0:
                            pro_k = sys.float_info.min
                        log_pro.append(pro_k)

                    pro_new_k = alpha*(len(D))
                    i = 0
                    for biterm in words:
                        for j in range(doc_biterm[docID][biterm]):
                            pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                            i += 1

                    if pro_new_k == 0:
                        pro_new_k = sys.float_info.min        
                    log_pro.append(pro_new_k)

                    sum_pro=sum(log_pro)

                    normalized_posterior = [i/sum_pro for i in log_pro]    
                    select_k = None
                    if iter == (total_iter - 1):
                        select_k = normalized_posterior.index(max(normalized_posterior))

                    else:
                        select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                    if select_k == len(Topics):
                        choose_k = np.max(Topics) + 1
                    else:
                        choose_k = Topics[select_k]

                    D.add(docID)
                    docID_assign_z[docID] = choose_k
                    if choose_k not in m_z.keys():
                        m_z[choose_k] = set()
                    m_z[choose_k].add(docID)
                    for biterm in words:
                        for word in biterm.split(','):
                            if choose_k not in n_w.keys():
                                n_w[choose_k] = 0
                            if choose_k not in n_z.keys():
                                n_z[choose_k] = {}
                            if word not in n_z[choose_k].keys():
                                n_z[choose_k][word] = 0
                            n_z[choose_k][word] += 1
                            n_w[choose_k] += 1
                            V.add(word)
                    if choose_k not in Topics:
                        Topics.append(choose_k)

                count_k = []
                for k in Topics:
                    if k in m_z.keys() and len(m_z[k]) == 0:
                        m_z.pop(k, None)
                        n_z.pop(k, None)
                        n_w.pop(k, None)
                        count_k.append(k)
                for k in count_k:
                    Topics.remove(k)   
            if iter == 9:
                from sklearn.metrics.cluster import normalized_mutual_info_score
                nmi_sample = []
                nmi_result = []
                for key, value in news_labels.items():
                    if key < end and key >= start:
                        nmi_sample.append(value)
                        nmi_result.append(docID_assign_z[key])
                NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
                topic_batch[batch_i] = len(Topics)
                #print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
        if batch_i != 1:
            start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    
    alpha_NMI[alpha] = np.mean(list(NMI_batch.values()))
    alpha_topics[alpha] = int(np.mean(list(topic_batch.values())))
    print("alpha ",alpha, " NMI: ", alpha_NMI[alpha])
    print("alpha ",alpha, " Topics: ", alpha_topics[alpha])

alpha  0.1  NMI:  0.8453845540783476
alpha  0.1  Topics:  81
alpha  0.2  NMI:  0.8486100535419043
alpha  0.2  Topics:  87
alpha  0.3  NMI:  0.84942774143682
alpha  0.3  Topics:  89
alpha  0.4  NMI:  0.8474128102136836
alpha  0.4  Topics:  91
alpha  0.5  NMI:  0.8460112095273967
alpha  0.5  Topics:  95
alpha  0.6  NMI:  0.8447677521639362
alpha  0.6  Topics:  94
alpha  0.7  NMI:  0.8489684822857698
alpha  0.7  Topics:  97
alpha  0.8  NMI:  0.8485642574705066
alpha  0.8  Topics:  98
alpha  0.9  NMI:  0.8441177310998838
alpha  0.9  Topics:  99
alpha  1.0  NMI:  0.8439768970506967
alpha  1.0  Topics:  101


In [9]:
print(alpha_NMI.values())
print(alpha_topics.values())

dict_values([0.8453845540783476, 0.8486100535419043, 0.84942774143682, 0.8474128102136836, 0.8460112095273967, 0.8447677521639362, 0.8489684822857698, 0.8485642574705066, 0.8441177310998838, 0.8439768970506967])
dict_values([81, 87, 89, 91, 95, 94, 97, 98, 99, 101])
